In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import warnings

warnings.filterwarnings('ignore')

In [7]:
file_name = f'./dbs/intermediate/stops_1.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)


In [4]:
df_2 = df[['uid','cluster','cluster_lat','cluster_lng']]
df_2.drop_duplicates(['uid','cluster'], inplace=True)
df_2 = df_2.reset_index()

In [5]:
building_point = pd.read_parquet(f'./dbs/intermediate/building_point.parquet')

building_metadata = pd.read_csv(f'./dbs/building_purpose_codes.csv', encoding='ISO-8859-1', sep=';')


building_point = pd.merge(building_point, building_metadata[['Code', 'Purpose of building']], left_on='ANDAMAL_1',
                          right_on='Code', how='left')

building_point.reset_index(inplace=True)


In [6]:
from sklearn.neighbors import BallTree


def building_query_radius(clusters, buildings):

    #convert degree to radian
    buildings['lat_rad'] = buildings.apply(lambda row: np.deg2rad(row['lat']), axis=1)
    buildings['lng_rad'] = buildings.apply(lambda row: np.deg2rad(row['lng']), axis=1)

    clusters['lat_rad'] = clusters.apply(lambda row: np.deg2rad(row['cluster_lat']), axis=1)
    clusters['lng_rad'] = clusters.apply(lambda row: np.deg2rad(row['cluster_lng']), axis=1)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(clusters[['lat_rad','lng_rad']])
    right_radians = np.array(buildings[['lat_rad','lng_rad']])

    tree = BallTree(right_radians, leaf_size=20, metric='haversine')
    kms_per_radian = 6371.0088

    # define epsilon as 0.1 kilometers, converted to radians for use by haversine
    epsilon = 0.1 / kms_per_radian

    is_within, distances = tree.query_radius(left_radians, r=epsilon, count_only=False, return_distance=True)

    return is_within




In [7]:
is_within = building_query_radius(df_2, building_point)

#distances = distances*6371.0088

In [8]:
Purpose_of_building = building_point["Purpose of building"].unique().tolist()

In [9]:
building_point = building_point[["Purpose of building"]]
building_point = building_point.to_numpy()

In [10]:
df_building_index = pd.DataFrame(is_within, columns=['building_index'])
df_building_index[Purpose_of_building] = 0

In [11]:
def select_count(data, build):
    selected_build = build[data['building_index']]
    unique, counts = np.unique(selected_build, return_counts=True)
    data.loc[unique] = counts

    return data





In [12]:
df_split = np.array_split(df_building_index, 40)

for i in range(0,40):
    df_split[i] = df_split[i].apply(lambda x: select_count(x, building_point), axis=1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [13]:
df_building_index = pd.concat(df_split)

In [14]:
df_building_index['sum'] = df_building_index[Purpose_of_building].sum(axis=1)

In [15]:
df_building_index.loc[:,Purpose_of_building] = df_building_index.loc[:,Purpose_of_building].div(df_building_index['sum'], axis=0)

In [16]:
df_building_index.iloc[:,1:] = df_building_index.iloc[:,1:].fillna(0)

In [17]:
df_2 = df_2.join(df_building_index)

In [18]:

df = pd.merge(df, df_2[['uid', 'cluster', 'building_index',  'Residential', 'Industrial', 'Other building', 'Activity', 'Public', 'Agricultural', 'sum']], on=['uid', 'cluster'], how='left' )

In [20]:
df.to_parquet(f'./dbs/intermediate/stops_2.parquet')
